In [1]:
import setGPU
from tensorflow.keras.datasets import cifar10
from kerastuner import HyperModel
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten
from kerastuner.tuners import (BayesianOptimization, Hyperband, RandomSearch)
import tensorflow as tf

from pathlib import Path
from loguru import logger
import time

setGPU: Setting GPU to: 0


/home/vuu/anaconda3/envs/tfpy3_exp/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/vuu/anaconda3/envs/tfpy3_exp/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/vuu/anaconda3/envs/tfpy3_exp/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/vuu/anaconda3/envs/tfpy3_exp/lib/python3

In [2]:
INPUT_SHAPE = (32, 32, 3)
NUM_CLASSES = 10
SEED = 1
MAX_TRIALS = 20
EXECUTION_PER_TRIAL = 2
HYPERBAND_MAX_EPOCHS = 40
BAYESIAN_NUM_INITIAL_POINTS = 1
N_EPOCH_SEARCH = 40

In [3]:
def load_data():
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    x_train = x_train.astype("float32")/255.0
    x_test = x_test.astype("float32")/255.0

    return x_test, x_train, y_test, y_train

In [4]:
def set_gpu_config():
    # Set up GPU config
    logger.info("Setting up GPU if found")
    physical_devices = tf.config.experimental.list_physical_devices("GPU")
    if physical_devices:
        for device in physical_devices:
            tf.config.experimental.set_memory_growth(device, True)

In [5]:
class CNNHyperModel(HyperModel):
    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes
        
    def build(self, hp):
        model = keras.Sequential()
        model.add(Conv2D(filters = 16, kernel_size=3, activation='relu', input_shape=self.input_shape))
        model.add(Conv2D(filters=16, activation='relu', kernel_size=3))
        model.add(MaxPooling2D(pool_size=2))
        model.add(Dropout(rate=hp.Float('dropout_1', min_value=0.0, max_value=0.5, default=0.25, step=0.05,)))
        model.add(Conv2D(filters=32, kernel_size=3, activation='relu'))
        model.add(Conv2D(filters=hp.Choice("num_filters", values=[32, 64], default=64,), activation='relu', kernel_size=3))
        model.add(MaxPooling2D(pool_size=2))
        model.add(Dropout(rate=hp.Float("dropout_2", min_value=0.0, max_value=0.5, default=0.25, step=0.05,)))
        model.add(Flatten())
        model.add(Dense(units=hp.Int("units", min_value=32, max_value=512, step=32, default=128), activation=
                        hp.Choice("dense_activation", values=['relu', 'tanh', "sigmoid"], default='relu'),))
        model.add(Dropout(rate=hp.Float("dropout_3", min_value=0.0, max_value=0.5, default=0.25, step=0.05)))
        model.add(Dense(self.num_classes, activation="softmax"))
        model.compile(optimizer=keras.optimizers.Adam(hp.Float("learning_rate", min_value=1e-4, max_value=1e-2, sampling='LOG', default=1e-3)),
                     loss="sparse_categorical_crossentropy", metrics=['accuracy'])
        return model

In [6]:
def define_tuners(hypermodel, directory, project_name):
    random_tuner = RandomSearch(hypermodel, objective='val_acc', seed=SEED, max_trials=MAX_TRIALS, executions_per_trial=EXECUTION_PER_TRIAL,
                                directory=f"{directory}_random_search", project_name=project_name)
    hyperband_tuner = Hyperband(hypermodel, max_epochs=HYPERBAND_MAX_EPOCHS, objective="val_acc", seed=SEED, executions_per_trial=EXECUTION_PER_TRIAL,
        directory=f"{directory}_hyperband", project_name=project_name,)
    
    bayesian_tuner = BayesianOptimization(hypermodel, objective='val_acc',seed=SEED, num_initial_points=BAYESIAN_NUM_INITIAL_POINTS,
        max_trials=MAX_TRIALS, directory=f"{directory}_bayesian", project_name=project_name)
    return [random_tuner, hyperband_tuner, bayesian_tuner]

In [7]:
def tuner_evaluation(tuner, x_test, x_train, y_test, y_train):
    set_gpu_config()

    # Overview of the task
    tuner.search_space_summary()

    # Performs the hyperparameter tuning
    logger.info("Start hyperparameter tuning")
    search_start = time.time()
    tuner.search(x_train, y_train, epochs=N_EPOCH_SEARCH, validation_split=0.1)
    search_end = time.time()
    elapsed_time = search_end - search_start

    # Show a summary of the search
    tuner.results_summary()

    # Retrieve the best model.
    best_model = tuner.get_best_models(num_models=1)[0]

    # Evaluate the best model.
    loss, accuracy = best_model.evaluate(x_test, y_test)
    return elapsed_time, loss, accuracy

In [8]:
def run_hyper_parameter_training():
    x_test, x_train, y_test, y_train = load_data()
    
    #print(x_test.shape, x_train.shape, y_test.shape, y_train.shape)
    hypermodel = CNNHyperModel(input_shape=INPUT_SHAPE, num_classes = NUM_CLASSES)
    output_dir = Path("../../outputs/cifar10/")
    
    tuners = define_tuners(hypermodel, directory=output_dir, project_name="simple_cnn_tuning")
    
    results = []
    for tuner in tuners:
        elapsed_time, loss, accuracy = tuner_evaluation(tuner, x_test, x_train, y_test, y_train)
        logger.info(f"Elapsed time = {elapsed_time:10.4f} s, accuracy = {accuracy}, loss = {loss}")
        results.append([elapsed_time, loss, accuracy])
    logger.info(results)

In [ ]:
run_hyper_parameter_training()

INFO:tensorflow:Reloading Oracle from existing project ../../outputs/cifar10_random_search/simple_cnn_tuning/oracle.json
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


2020-04-14 16:07:28.572 | INFO     | __main__:set_gpu_config:3 - Setting up GPU if found


2020-04-14 16:07:30.163 | INFO     | __main__:tuner_evaluation:8 - Start hyperparameter tuning


Train on 45000 samples, validate on 5000 samples
Epoch 1/40
45000/45000 [==============================] - ETA: 30:55 - loss: 2.7258 - acc: 0.06 - ETA: 3:33 - loss: 2.5119 - acc: 0.1146 - ETA: 1:56 - loss: 2.4854 - acc: 0.106 - ETA: 1:25 - loss: 2.4731 - acc: 0.109 - ETA: 1:07 - loss: 2.4695 - acc: 0.106 - ETA: 57s - loss: 2.4649 - acc: 0.102 - ETA: 49s - loss: 2.4488 - acc: 0.10 - ETA: 44s - loss: 2.4376 - acc: 0.10 - ETA: 39s - loss: 2.4311 - acc: 0.10 - ETA: 36s - loss: 2.4293 - acc: 0.10 - ETA: 33s - loss: 2.4234 - acc: 0.10 - ETA: 31s - loss: 2.4146 - acc: 0.10 - ETA: 29s - loss: 2.4101 - acc: 0.10 - ETA: 28s - loss: 2.4030 - acc: 0.10 - ETA: 26s - loss: 2.3937 - acc: 0.11 - ETA: 25s - loss: 2.3873 - acc: 0.11 - ETA: 24s - loss: 2.3776 - acc: 0.11 - ETA: 23s - loss: 2.3664 - acc: 0.12 - ETA: 22s - loss: 2.3599 - acc: 0.12 - ETA: 21s - loss: 2.3473 - acc: 0.12 - ETA: 21s - loss: 2.3348 - acc: 0.13 - ETA: 20s - loss: 2.3243 - acc: 0.13 - ETA: 19s - loss: 2.3163 - acc: 0.14 - ETA: 19

Train on 45000 samples, validate on 5000 samples
Epoch 1/40
45000/45000 [==============================] - ETA: 3:25 - loss: 2.6119 - acc: 0.125 - ETA: 27s - loss: 2.6775 - acc: 0.087 - ETA: 19s - loss: 2.6084 - acc: 0.09 - ETA: 16s - loss: 2.5843 - acc: 0.09 - ETA: 15s - loss: 2.5492 - acc: 0.08 - ETA: 14s - loss: 2.5263 - acc: 0.08 - ETA: 13s - loss: 2.5112 - acc: 0.09 - ETA: 13s - loss: 2.4971 - acc: 0.09 - ETA: 12s - loss: 2.4842 - acc: 0.09 - ETA: 12s - loss: 2.4807 - acc: 0.09 - ETA: 12s - loss: 2.4720 - acc: 0.09 - ETA: 11s - loss: 2.4673 - acc: 0.09 - ETA: 11s - loss: 2.4577 - acc: 0.09 - ETA: 11s - loss: 2.4500 - acc: 0.09 - ETA: 11s - loss: 2.4444 - acc: 0.09 - ETA: 11s - loss: 2.4408 - acc: 0.09 - ETA: 10s - loss: 2.4381 - acc: 0.09 - ETA: 10s - loss: 2.4360 - acc: 0.09 - ETA: 10s - loss: 2.4297 - acc: 0.09 - ETA: 10s - loss: 2.4255 - acc: 0.10 - ETA: 10s - loss: 2.4208 - acc: 0.10 - ETA: 10s - loss: 2.4154 - acc: 0.10 - ETA: 10s - loss: 2.4116 - acc: 0.10 - ETA: 10s - loss:

...

45000/45000 [==============================] - ETA: 7s - loss: 1.3726 - acc: 0.506 - ETA: 7s - loss: 1.3708 - acc: 0.507 - ETA: 7s - loss: 1.3699 - acc: 0.507 - ETA: 7s - loss: 1.3702 - acc: 0.507 - ETA: 7s - loss: 1.3695 - acc: 0.508 - ETA: 7s - loss: 1.3688 - acc: 0.508 - ETA: 7s - loss: 1.3679 - acc: 0.509 - ETA: 7s - loss: 1.3653 - acc: 0.510 - ETA: 7s - loss: 1.3654 - acc: 0.510 - ETA: 6s - loss: 1.3646 - acc: 0.510 - ETA: 6s - loss: 1.3638 - acc: 0.511 - ETA: 6s - loss: 1.3634 - acc: 0.511 - ETA: 6s - loss: 1.3640 - acc: 0.511 - ETA: 6s - loss: 1.3636 - acc: 0.511 - ETA: 6s - loss: 1.3631 - acc: 0.511 - ETA: 6s - loss: 1.3619 - acc: 0.511 - ETA: 6s - loss: 1.3630 - acc: 0.511 - ETA: 6s - loss: 1.3620 - acc: 0.511 - ETA: 6s - loss: 1.3625 - acc: 0.512 - ETA: 6s - loss: 1.3616 - acc: 0.512 - ETA: 6s - loss: 1.3613 - acc: 0.512 - ETA: 6s - loss: 1.3602 - acc: 0.512 - ETA: 6s - loss: 1.3591 - acc: 0.512 - ETA: 6s - loss: 1.3589 - acc: 0.513 - ETA: 6s - loss: 1.3593 - acc: 0.513 - ETA

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



45000/45000 [==============================] - ETA: 8s - loss: 0.6267 - acc: 0.776 - ETA: 8s - loss: 0.6282 - acc: 0.775 - ETA: 8s - loss: 0.6273 - acc: 0.775 - ETA: 8s - loss: 0.6240 - acc: 0.776 - ETA: 8s - loss: 0.6258 - acc: 0.775 - ETA: 8s - loss: 0.6265 - acc: 0.774 - ETA: 8s - loss: 0.6252 - acc: 0.775 - ETA: 8s - loss: 0.6253 - acc: 0.775 - ETA: 8s - loss: 0.6267 - acc: 0.775 - ETA: 8s - loss: 0.6268 - acc: 0.775 - ETA: 8s - loss: 0.6322 - acc: 0.774 - ETA: 8s - loss: 0.6323 - acc: 0.774 - ETA: 8s - loss: 0.6323 - acc: 0.774 - ETA: 8s - loss: 0.6299 - acc: 0.775 - ETA: 8s - loss: 0.6294 - acc: 0.775 - ETA: 8s - loss: 0.6303 - acc: 0.775 - ETA: 8s - loss: 0.6330 - acc: 0.774 - ETA: 8s - loss: 0.6315 - acc: 0.775 - ETA: 7s - loss: 0.6311 - acc: 0.775 - ETA: 7s - loss: 0.6318 - acc: 0.775 - ETA: 7s - loss: 0.6314 - acc: 0.775 - ETA: 7s - loss: 0.6302 - acc: 0.776 - ETA: 7s - loss: 0.6307 - acc: 0.776 - ETA: 7s - loss: 0.6304 - acc: 0.776 - ETA: 7s - loss: 0.6320 - acc: 0.776 - ETA

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



45000/45000 [==============================] - ETA: 10s - loss: 1.1424 - acc: 0.58 - ETA: 10s - loss: 1.1359 - acc: 0.59 - ETA: 10s - loss: 1.1470 - acc: 0.59 - ETA: 10s - loss: 1.1652 - acc: 0.59 - ETA: 10s - loss: 1.1599 - acc: 0.59 - ETA: 10s - loss: 1.1543 - acc: 0.59 - ETA: 10s - loss: 1.1560 - acc: 0.59 - ETA: 10s - loss: 1.1574 - acc: 0.59 - ETA: 9s - loss: 1.1503 - acc: 0.5948 - ETA: 9s - loss: 1.1408 - acc: 0.600 - ETA: 9s - loss: 1.1446 - acc: 0.595 - ETA: 9s - loss: 1.1460 - acc: 0.593 - ETA: 9s - loss: 1.1509 - acc: 0.592 - ETA: 9s - loss: 1.1520 - acc: 0.593 - ETA: 9s - loss: 1.1493 - acc: 0.595 - ETA: 9s - loss: 1.1494 - acc: 0.597 - ETA: 9s - loss: 1.1449 - acc: 0.597 - ETA: 9s - loss: 1.1415 - acc: 0.599 - ETA: 9s - loss: 1.1394 - acc: 0.600 - ETA: 9s - loss: 1.1343 - acc: 0.601 - ETA: 9s - loss: 1.1322 - acc: 0.601 - ETA: 9s - loss: 1.1304 - acc: 0.602 - ETA: 9s - loss: 1.1303 - acc: 0.603 - ETA: 9s - loss: 1.1315 - acc: 0.601 - ETA: 9s - loss: 1.1306 - acc: 0.601 - ET